In [1]:
import Pkg; Pkg.add("Tables")
using CSV, Tables
using JuMP
using Gurobi

Pkg.add("JSON3")
using JSON
using JSON3
using HTTP

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


In [2]:
const GRB_ENV = Gurobi.Env(output_flag=1);

Set parameter Username
Set parameter LicenseID to value 2595637
Academic license - for non-commercial use only - expires 2025-12-05


#### Load in Data

In [3]:
all_classes = open("data/all_classes_info.json", "r") do file 
    JSON3.read(file)
end

R = CSV.File("data/ratings.csv",header=0) |> Tables.matrix;
H = CSV.File("data/hours.csv",header=0) |> Tables.matrix;
U = CSV.File("data/units.csv",header=0) |> Tables.matrix;

vars = CSV.File("data/variables.csv",header=0) |> Tables.matrix;
n = vars[1];
S = vars[2];

In [4]:
random_class = all_classes[rand(1:length(all_classes))]

JSON3.Object{Base.CodeUnits{UInt8, String}, SubArray{UInt64, 1, Vector{UInt64}, Tuple{UnitRange{Int64}}, true}} with 26 entries:
  :rating             => 6
  :has_final          => false
  :description        => "Analyzes the factors involved in the digital transfor…
  :offered_fall       => false
  :offered_spring     => true
  :instructors        => ["M. Jesus Saenz"]
  :out_of_class_hours => 5.35
  :total_units        => 6
  :related_subjects   => ["SCM.275", "SCM.290", "SCM.265", "SCM.500", "SCM.261"…
  :pdf_option         => false
  :in_class_hours     => 3.1
  :is_half_class      => false
  :level              => "G"
  :url                => "http://student.mit.edu/catalog/mSCMa.html#SCM.294"
  :subject_id         => "SCM.294"
  :title              => "Digital Supply Chain Transformation"
  :lab_units          => 0
  :design_units       => 0
  :public             => true
  ⋮                   => ⋮

In [5]:
subjectid_to_index = Dict(class[:subject_id] => ind for (ind, class) in enumerate(all_classes))
index_to_subjectid = Dict(ind => class[:subject_id] for (ind, class) in enumerate(all_classes))

Dict{Int64, String} with 4704 entries:
  4700 => "WGS.301"
  4576 => "STS.022"
  2288 => "21G.905"
  1703 => "18.C20"
  1956 => "20.101"
  2350 => "21H.226"
  3406 => "6.2060"
  2841 => "24.234"
  2876 => "24.906"
  687  => "12.751"
  185  => "1.THG"
  1090 => "15.518"
  2015 => "20.560"
  3293 => "5.00"
  1704 => "18.S096"
  3220 => "4.608"
  422  => "11.353"
  1266 => "16.004"
  183  => "1.S992"
  ⋮    => ⋮

#### Defining the Basic Optimization Problem

In [6]:
function create_model(env) 
    return  Model(() -> Gurobi.Optimizer(env))
end
model1 = create_model(GRB_ENV)

A JuMP Model
├ solver: Gurobi
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

In [7]:
function add_course_semester_variables!(model::JuMP.Model)
    @variable(model, [1:n, 1:S], Bin)
end
model_var = add_course_semester_variables!(model1)

4704×8 Matrix{VariableRef}:
 _[1]     _[4705]  _[9409]   _[14113]  _[18817]  _[23521]  _[28225]  _[32929]
 _[2]     _[4706]  _[9410]   _[14114]  _[18818]  _[23522]  _[28226]  _[32930]
 _[3]     _[4707]  _[9411]   _[14115]  _[18819]  _[23523]  _[28227]  _[32931]
 _[4]     _[4708]  _[9412]   _[14116]  _[18820]  _[23524]  _[28228]  _[32932]
 _[5]     _[4709]  _[9413]   _[14117]  _[18821]  _[23525]  _[28229]  _[32933]
 _[6]     _[4710]  _[9414]   _[14118]  _[18822]  _[23526]  _[28230]  _[32934]
 _[7]     _[4711]  _[9415]   _[14119]  _[18823]  _[23527]  _[28231]  _[32935]
 _[8]     _[4712]  _[9416]   _[14120]  _[18824]  _[23528]  _[28232]  _[32936]
 _[9]     _[4713]  _[9417]   _[14121]  _[18825]  _[23529]  _[28233]  _[32937]
 _[10]    _[4714]  _[9418]   _[14122]  _[18826]  _[23530]  _[28234]  _[32938]
 ⋮                                               ⋮                   
 _[4696]  _[9400]  _[14104]  _[18808]  _[23512]  _[28216]  _[32920]  _[37624]
 _[4697]  _[9401]  _[14105]  _[18809]  _[235

In [8]:
function set_objective_func(model, dVar, α= .8)
    @objective(model, Min, sum( sqrt(s)*(α*H[i]*dVar[i,s] - (1-α)*R[i]*dVar[i,s] + 5*dVar[i,s])  for i=1:n,s=1:S));
end
set_objective_func(model1, model_var, 1)

18.689999999999998 _[1] + 26.431651480753146 _[4705] + 32.37202959346231 _[9409] + 37.379999999999995 _[14113] + 41.792110499471065 _[18817] + 45.78096329261759 _[23521] + 49.4490920037972 _[28225] + 52.86330296150629 _[32929] + 15.68 _[2] + 22.17486865801013 _[4706] + 27.158556662679995 _[9410] + 31.36 _[14114] + 35.0615458871967 _[18818] + 38.40799916684023 _[23522] + 41.48538055749278 _[28226] + 44.34973731602026 _[32930] + 18.689999999999998 _[3] + 26.431651480753146 _[4707] + 32.37202959346231 _[9411] + 37.379999999999995 _[14115] + 41.792110499471065 _[18819] + 45.78096329261759 _[23523] + 49.4490920037972 _[28227] + 52.86330296150629 _[32931] + 11 _[4] + 15.556349186104047 _[4708] + 19.05255888325765 _[9412] + 22 _[14116] + 24.596747752497688 _[18820] + 26.944387170614956 _[23524] + [[...37572 terms omitted...]] + 23.209480821422954 _[14109] + 26.8 _[18813] + 29.963310898497184 _[23517] + 32.82316255329459 _[28221] + 35.45306756826552 _[32925] + 37.90092347159895 _[37629] + 13.6

In [9]:
function add_general_requirements(model, dVar) 
    # Full-time; at least 36 units per semester
    @constraint(model, [s in 1:S], sum(U[i]*dVar[i, s] for i=1:n) >= 36);

    # Wellbeing: no more than 112 hours per week
    @constraint(model, [s in 1:S], sum(H[i]*dVar[i, s] for i=1:n) <= 112);

    # Not repeatable: cannot take a class again
    @constraint(model, [i in 1:n], sum(dVar[i, t] for t=1:S) <= 1);

    # Taken correct semester
    @constraint(model, [s in 1:2:S, i in 1:n], dVar[i, s] <= all_classes[i][:offered_fall]);
    @constraint(model, [s in 2:2:S, i in 1:n], dVar[i, s] <= all_classes[i][:offered_spring]);

    # 2 CI-H or HW
    @constraint(model, sum(
        (get(all_classes[i], :communication_requirement, "") in ["CI-H", "CI-HW"]) * dVar[i, s] for i=1:n, s=1:S) >= 2);
    
    # 8 HASS
    @constraint(model, sum(
        (get(all_classes[i], :hass_attribute, "") in ["HASS-H", "HASS-A", "HASS-S"]) * dVar[i, s] for i=1:n, s=1:S) >= 8);

    # 1 of each HASS-H, A, S
    @constraint(model, [hass_type in ["HASS-H", "HASS-A", "HASS-S"]], sum(
        (get(all_classes[i], :hass_attribute, "") in [hass_type]) * dVar[i, s] for i=1:n, s=1:S) >= 1);
end
add_general_requirements(model1, model_var)

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, ["HASS-H", "HASS-A", "HASS-S"]
And data, a 3-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 _[194] + _[300] + _[308] + _[309] + _[310] + _[311] + _[315] + _[329] + _[335] + _[343] + _[344] + _[346] + _[348] + _[1398] + _[1399] + _[1400] + _[1403] + _[1404] + _[1406] + _[2039] + _[2040] + _[2043] + _[2092] + _[2095] + _[2096] + _[2100] + _[2102] + _[2103] + _[2104] + _[2105] + [[...3844 terms omitted...]] + _[37526] + _[37535] + _[37538] + _[37542] + _[37543] + _[37584] + _[37585] + _[37586] + _[37587] + _[37588] + _[37591] + _[37592] + _[37593] + _[37596] + _[37599] + _[37601] + _[37604] + _[37609] + _[37612] + _[37613] + _[37614] + _[37615]

#### Adding Institute Reqs ####

In [10]:
chem_gir_ids = ["3.091", "5.111", "5.112"]
phys_1_gir_ids = ["8.01", "8.011", "8.012", "8.01L"]
phys_2_gir_ids = ["8.02", "8.021", "8.022"]
math_1_gir_ids = ["18.01", "18.01A"]
math_2_gir_ids = ["18.02", "18.02A", "18.022"]
bio_gir_ids = ["7.012", "7.013", "7.014", "7.015", "7.016"]
gir_ids = [chem_gir_ids, phys_1_gir_ids, phys_2_gir_ids, math_1_gir_ids, math_2_gir_ids, bio_gir_ids]
gir_inds_types = [[findfirst(x -> x[:subject_id] == gir_id, all_classes) for gir_id in gir_type] for gir_type in gir_ids]

6-element Vector{Vector{Int64}}:
 [2963, 3304, 3305]
 [3857, 3858, 3859, 3860]
 [3861, 3862, 3863]
 [1537, 1538]
 [1539, 1541, 1540]
 [3753, 3754, 3755, 3756, 3757]

In [11]:
function add_gir_constraints(model, dVar)
    @constraint(model, [gir_type in gir_inds_types], sum(dVar[gir_ind, s] for gir_ind in gir_type, s in 1:S) == 1);
end
add_gir_constraints(model1, model_var)

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, [[2963, 3304, 3305], [3857, 3858, 3859, 3860], [3861, 3862, 3863], [1537, 1538], [1539, 1541, 1540], [3753, 3754, 3755, 3756, 3757]]
And data, a 6-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 _[2963] + _[3304] + _[3305] + _[7667] + _[8008] + _[8009] + _[12371] + _[12712] + _[12713] + _[17075] + _[17416] + _[17417] + _[21779] + _[22120] + _[22121] + _[26483] + _[26824] + _[26825] + _[31187] + _[31528] + _[31529] + _[35891] + _[36232] + _[36233] = 1
 _[3857] + _[3858] + _[3859] + _[3860] + _[8561] + _[8562] + _[8563] + _[8564] + _[13265] + _[13266] + _[13267] + _[13268] + _[17969] + _[17970] + _[17971] + _[17972] + _[22673] + _[22674] + _[22675] + _

#### Adding 6-4 Constraints ####

In [43]:
function get_major_requirements(major::String)
    # Store the URL
    url = "https://fireroad.mit.edu/requirements/get_json/major"*major*"/"

    # Send the request and get the response
    reqs = HTTP.get(url)

    # Parse the JSON response
    datafile = JSON.parse(String(reqs.body))
    return datafile
end

df = get_major_requirements("6-4")

Dict{String, Any} with 7 entries:
  "short-title"     => "6-4"
  "medium-title"    => "6-4 Major"
  "title-no-degree" => "Artificial Intelligence and Decision Making"
  "reqs"            => Any[Dict{String, Any}("connection-type"=>"all", "thresho…
  "list-id"         => "major6-4.reql"
  "title"           => "Bachelor of Science in Artificial Intelligence and Deci…
  "desc"            => "The Bachelor of Science in Artificial Intelligence and …

In [44]:
# Create a mapping for comparison types
type_mapping = Dict("LT" => "lt", "GT" => "gt", "GTE" => "geq", "LTE" => "leq")

# Function to recursively go through requirements and create vector of major requirements
function create_major_requirements(data, connector=false, cutoff=1, ineq="geq")
    mandatory = []
    all_reqs = []
    together = false
    comb_req = []
    
    for section in data["reqs"]
        ineq = "geq"
        cutoff = 1
        
        # Base case: if there's a "req" key in the section
        if haskey(section, "req")
            if connector
                push!(mandatory, section["req"])
                continue
            else
                push!(all_reqs, section["req"])
                continue
            end
        end

        # Recursive case: checking for special thresholds
        if haskey(section, "distinct-threshold")
            cutoff = section["distinct-threshold"]["cutoff"]
            ineq = type_mapping[section["distinct-threshold"]["type"]]
            if haskey(section, "threshold") && section["threshold"]["cutoff"] != cutoff
                together = true
                total_cutoff = section["threshold"]["cutoff"]
                total_ineq = type_mapping[section["threshold"]["type"]]
            end
        elseif haskey(section, "threshold")
            cutoff = section["threshold"]["cutoff"]
            ineq = type_mapping[section["threshold"]["type"]]
        end
        
        # Building based on connection types
        if section["connection-type"] == "all"
            next = create_major_requirements(section, true)
            if together
                comb_req = vcat(comb_req, reduce(vcat, getindex.(next, 1)))
            else
                append!(all_reqs, next)
            end
        elseif section["connection-type"] == "any"
            next = create_major_requirements(section)
            if together
                comb_req = vcat(comb_req, reduce(vcat, getindex.(next, 1)))
            else
                one_req = (next, ineq, cutoff)
                push!(all_reqs, one_req)
            end
        else
            return create_major_requirements(section)
        end
        
        #for thresholds with distinct-thresholds
        if together
            push!(all_reqs, (comb_req, total_ineq, total_cutoff))
        end
    end

    if !isempty(mandatory)
        push!(all_reqs, (mandatory, "eq", length(mandatory)))
    end
    
    return all_reqs
end

# Call the function
major_requirements = create_major_requirements(df)

10-element Vector{Any}:
 (Any["6.100A", "6.100L"], "geq", 1)
 (Any["6.S084", "18.C06", "18.06"], "geq", 1)
 (Any["6.3700", "6.3800", "18.05"], "geq", 1)
 (Any["6.1200", "6.1010", "6.1210"], "eq", 3)
 (Any[(Any["6.3720", "6.3900", (Any["6.C01", "6.S052"], "eq", 2), (Any["6.C51", "6.S952"], "eq", 2), "6.S059"], "geq", 1), (Any["6.3000", "6.4110", "6.4400"], "geq", 1), (Any["6.3100", "6.4110", "6.7201"], "geq", 1), (Any["6.1220", "6.4400", "6.7201"], "geq", 1), (Any["6.3260", "6.3950", "6.4120", "6.4590", "6.C35", "6.S041", "9.660"], "geq", 1)], "geq", 5)
 (Any["6.1800", "6.2040", "6.2060", "6.2061", "6.2220", "6.2221", "6.2370", "6.2600", "6.4200", "6.4210", "6.4590", "6.4860", "6.4880", "6.8301", "6.8611", "6.9030", "6.UAR", "6.UAT"], "geq", 2)
 (Any["6.3900", "6.3950", "6.4590", "6.8301", "6.8611"], "geq", 1)
 (Any["6.4200", "6.4210", "6.8301", "6.8611"], "geq", 1)
 (Any["18.404", "6.3730", "6.4210", "6.5151", "6.5831", "6.7411", "6.7930", "6.8300", "6.8301", "6.8371", "6.8611", "6.870

In [46]:
# Helper function to map the inequality string to a JuMP constraint
function create_inequality_constraint(model, expr, ineq::String, value)
    if ineq == "geq"
        @constraint(model, expr >= value)
    elseif ineq == "eq"
        @constraint(model, expr == value)
    elseif ineq == "leq"
        @constraint(model, expr <= value)
    else
        error("Unknown inequality type: $ineq")
    end
end

# Main function to create JuMP constraints based on the requirements vector
function add_requirements_constraints(model::Model, requirements::Vector, dVar, bin_var::Bool=false, group::Bool=false)
    outer_group = Vector{Any}()
    for req in requirements
        courses, ineq, cutoff = req

        # If the course requirement is a nested structure (i.e., a vector of other requirements)
        if any(x -> isa(x, Tuple), courses)
            # Recursively handle the nested requirements
            solos = Vector{Any}()
            for nested_req in courses
                if isa(nested_req, Tuple)
                    temps = add_requirements_constraints(model, [nested_req], dVar, true)
                    append!(outer_group, temps)
                else
                    push!(solos, nested_req)
                end
            end
            if !isempty(solos)
                temps = add_requirements_constraints(model, [(solos, "geq", 1)], dVar, true)
                append!(outer_group, temps)
            end
            if !isempty(outer_group)
                final_tup = (outer_group, ineq, cutoff)
                add_requirements_constraints(model, [final_tup], dVar, false, true)
                outer_group = [@variable(model, binary=true)]
            end
        else
            # Retrieve the variables for each course in the list
            if group
                course_inds = courses
                course_expr = sum(course_inds)
            else
                course_inds = [subjectid_to_index[course] for course in courses if haskey(subjectid_to_index, course)]
                if isempty(course_inds)
                    continue
                end
                course_expr = sum(dVar[course_ind, s] for course_ind in course_inds, s in 1:S)
            end

            if bin_var
                inner_group = @variable(model, binary=true)
                push!(outer_group, inner_group)
                # Create the appropriate inequality constraint
                new_cutoff = cutoff * inner_group
                create_inequality_constraint(model, course_expr, ineq, new_cutoff)
            else
                # Create the appropriate inequality constraint
                create_inequality_constraint(model, course_expr, ineq, cutoff)
            end
        end
    end
    return outer_group
end

real_major_requirements = [(major_requirements, "eq", length(major_requirements))]
add_requirements_constraints(model1, real_major_requirements, model_var)

1-element Vector{VariableRef}:
 _[96779]

In [15]:
function add_major_requirements(major::String, model::Model, dVar)
    datafile = get_major_requirements(major)
    parsed_requirements = create_major_requirements(datafile)
    requirements = [(parsed_requirements, "eq", length(parsed_requirements))]
    add_requirements_constraints(model, requirements, dVar)
end

add_major_requirements (generic function with 1 method)

#### Adding Prereq constraints

In [16]:
raw_prerequisites = map(course -> get(course, "prerequisites", nothing), all_classes)

4704-element Vector{Union{Nothing, String}}:
 "GIR:CAL1, ((6.100A, 6.100B)/(6.100L, 16.C20))"
 nothing
 "GIR:CAL1"
 nothing
 nothing
 nothing
 nothing
 "GIR:CAL2"
 nothing
 "''Permission of instructor''"
 ⋮
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 "''One intermediate subject in French''/''permission of instructor''"
 nothing

In [17]:
abstract type ExprNode end

struct CourseNode <: ExprNode
    course::String
end

struct ANDNode <: ExprNode
    left::ExprNode
    right::ExprNode
end

struct ORNode <: ExprNode
    left::ExprNode
    right::ExprNode
end

struct PreReq
    root::CourseNode
    prereqs::Union{ExprNode, Nothing}
end 

In [19]:
function parse_prereq_string(input)
    if input === nothing
        return
    end
    input = replace(input, r"\s+" => "")
    tokens = collect(input)
    function parse(tokens::Vector{Char})::ExprNode
        stack = ExprNode[]
        buffer = IOBuffer()
        while !isempty(tokens)
            token = popfirst!(tokens)
            if token == '('
                node = parse(tokens)
                push!(stack, node)
            elseif token == ')'
                break
            elseif token == ','
                left = isempty(stack) ? CourseNode(String(take!(buffer))) : pop!(stack)
                right = parse(tokens)
                return ANDNode(left, right)
            elseif token == '/'
                left = isempty(stack) ? CourseNode(String(take!(buffer))) : pop!(stack)
                right = parse(tokens)
                return ORNode(left, right)
            else
                write(buffer, token)
            end
        end
        return isempty(stack) ? CourseNode(String(take!(buffer))) : stack[1]
    end
    return parse(tokens)
end

parse_prereq_string (generic function with 1 method)

In [20]:
function replace_gir_terms(input_str)
    if input_str === nothing
        return 
    end
    gir_map = Dict(
        "GIR:CHEM" => ["3.091", "5.111", "5.112"],
        "GIR:PHY1" => ["8.01", "8.011", "8.012", "8.01L"],
        "GIR:PHY2" => ["8.02", "8.021", "8.022"],
        "GIR:CAL1" => ["18.01", "18.01A"],
        "GIR:CAL2" => ["18.02", "18.02A", "18.022"],
        "GIR:BIOL" => ["7.012", "7.013", "7.014", "7.015", "7.016"]
    )
    pattern = r"GIR:\w+"
    replaced_str = replace(input_str, pattern => x -> begin
        term = match(pattern, x).match
        if haskey(gir_map, term)
            "(" * join(gir_map[term], "/") * ")"
        else
            term
        end
    end)
    return replaced_str
end

replace_gir_terms (generic function with 1 method)

In [21]:
prerequisites = map(prereq_str -> replace_gir_terms(prereq_str), raw_prerequisites)

4704-element Vector{Union{Nothing, String}}:
 "(18.01/18.01A), ((6.100A, 6.100B)/(6.100L, 16.C20))"
 nothing
 "(18.01/18.01A)"
 nothing
 nothing
 nothing
 nothing
 "(18.02/18.02A/18.022)"
 nothing
 "''Permission of instructor''"
 ⋮
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 "''One intermediate subject in French''/''permission of instructor''"
 nothing

In [22]:
prerequisite_expressions = []
for index in 1:length(prerequisites)
    course_id = CourseNode(index_to_subjectid[index])
    prereq_tree = parse_prereq_string(prerequisites[index])
    push!(prerequisite_expressions, PreReq(course_id, prereq_tree))
end
prerequisite_expressions

4704-element Vector{Any}:
 PreReq(CourseNode("1.00"), ANDNode(ORNode(CourseNode("18.01"), CourseNode("18.01A")), ORNode(ANDNode(CourseNode("6.100A"), CourseNode("6.100B")), ANDNode(CourseNode("6.100L"), CourseNode("16.C20")))))
 PreReq(CourseNode("1.000"), nothing)
 PreReq(CourseNode("1.001"), ORNode(CourseNode("18.01"), CourseNode("18.01A")))
 PreReq(CourseNode("1.005"), nothing)
 PreReq(CourseNode("1.007"), nothing)
 PreReq(CourseNode("1.008"), nothing)
 PreReq(CourseNode("1.009"), nothing)
 PreReq(CourseNode("1.010"), ORNode(CourseNode("18.02"), ORNode(CourseNode("18.02A"), CourseNode("18.022"))))
 PreReq(CourseNode("1.011"), nothing)
 PreReq(CourseNode("1.013"), CourseNode("''Permissionofinstructor''"))
 ⋮
 PreReq(CourseNode("WGS.274"), nothing)
 PreReq(CourseNode("WGS.275"), nothing)
 PreReq(CourseNode("WGS.276"), nothing)
 PreReq(CourseNode("WGS.277"), nothing)
 PreReq(CourseNode("WGS.301"), nothing)
 PreReq(CourseNode("WGS.303"), nothing)
 PreReq(CourseNode("WGS.310"), nothing)


In [23]:
function add_course_constraint(model::Model, dVar, course::CourseNode, sem::Int)
    index = get(subjectid_to_index, course.course, nothing)
    if index == nothing
        return 0  # Ignore unknown courses
    end
    if sem == 1
        return 0  # Cannot satisfy prerequisites in the first semester
    else
        return handle_prerequisite_sum(model, dVar, index, sem)
    end
end

function handle_prerequisite_sum(model::Model, dVar, index::Int, sem::Int)
    prereq_sum = sum(dVar[index, s] for s in 1:sem-1)
    binary_prereq_var = @variable(model, binary=true)
    @constraint(model, binary_prereq_var <= prereq_sum)
    @constraint(model, binary_prereq_var >=  prereq_sum / (sem-1))  # Ensure binary behavior
    return binary_prereq_var
end

function add_and_constraint(left_var, right_var, model::Model)
    and_var = @variable(model, binary=true)
    @constraint(model, and_var <= left_var)
    @constraint(model, and_var <= right_var)
    @constraint(model, and_var >= left_var + right_var - 1)
    return and_var
end

function add_or_constraint(left_var, right_var, model::Model)
    or_var = @variable(model, binary=true)
    @constraint(model, or_var >= left_var)
    @constraint(model, or_var >= right_var)
    @constraint(model, or_var <= left_var + right_var)
    return or_var
end

function add_and_or_constraints(model::Model, dVar, expr::ExprNode, sem::Int)
    if isa(expr, CourseNode)
        return add_course_constraint(model, dVar, expr, sem)
    elseif isa(expr, ANDNode)
        left_var = add_and_or_constraints(model, dVar, expr.left, sem)
        right_var = add_and_or_constraints(model, dVar, expr.right, sem)
        return add_and_constraint(left_var, right_var, model)
    elseif isa(expr, ORNode)
        left_var = add_and_or_constraints(model, dVar, expr.left, sem)
        right_var = add_and_or_constraints(model, dVar, expr.right, sem)
        return add_or_constraint(left_var, right_var, model)
    else
        error("Unknown expression type")
    end
end

function add_prereq_constraint(model, dVar, prereq_obj, sem)
    index = get(subjectid_to_index, prereq_obj.root.course, nothing)
    if index == nothing
        return 1  # Ignore unknown courses
    end
    if prereq_obj.prereqs === nothing
        return dVar[index, sem]
    else
        prereq_var = add_and_or_constraints(model, dVar, prereq_obj.prereqs, sem)
        @constraint(model, dVar[index, sem] <= prereq_var)
        return dVar[index, sem]
    end
end


add_prereq_constraint (generic function with 1 method)

In [24]:
function add_prereq_constraints(model, dVar) 
    for s in 1:S
        for prereq_expr in prerequisite_expressions
            add_prereq_constraint(model, dVar, prereq_expr, s)
        end
    end
end 

add_prereq_constraints(model1, model_var)

#### Optimizing the Model and Interpretation

In [25]:
optimize!(model1)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.6.0 23G93)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

Optimize a model with 211887 rows, 96708 columns and 682465 nonzeros
Model fingerprint: 0xa181ee44
Variable types: 0 continuous, 96708 integer (96708 binary)
Coefficient statistics:
  Matrix range     [1e-01, 8e+01]
  Objective range  [6e+00, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 197365 rows and 83355 columns
Presolve time: 0.82s
Presolved: 14522 rows, 13353 columns, 81569 nonzeros
Variable types: 0 continuous, 13353 integer (13353 binary)
Found heuristic solution: objective 1798.6119841
Performing another presolve...
Presolve removed 3988 rows and 775 columns
Presolve time: 0.22s

Root relaxation: objective 6.555688e+02, 292 iterations, 0.01 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl 

In [26]:
function interpret_course_nums(classes_id_per_sem)
    course_nums = [[all_classes[class_ind][:subject_id] for class_ind in sem] for sem in classes_id_per_sem]
    return course_nums
end

function interpret_course_attr(classes_id_per_sem, attr)
    courses_attr = [[get(all_classes[class_ind], Symbol(attr), "") for class_ind in sem] for sem in classes_id_per_sem]
    return courses_attr
end

function interpret_course_hours(classes_id_per_sem)
    course_hours = [[H[class_ind] for class_ind in sem] for sem in classes_id_per_sem]
    return course_hours
end

function interpret_course_units(classes_id_per_sem)
    course_units = [[U[class_ind] for class_ind in sem] for sem in classes_id_per_sem]
    return course_units
end

interpret_course_units (generic function with 1 method)

In [27]:
taken_classes_sem = [[ind for (ind, val) in enumerate(sem) if val == 1] for sem in eachcol(value.(model_var))]
interpret_course_nums(taken_classes_sem)

8-element Vector{Vector{String}}:
 ["18.01", "6.100A", "6.UAT", "7.012", "8.01L"]
 ["18.02", "6.100B", "6.1010", "6.1200"]
 ["18.06", "6.1210", "6.3700"]
 ["6.3260", "6.8301", "8.02"]
 ["6.3100", "6.3900", "6.4400"]
 ["11.013", "3.091", "STS.083"]
 ["21M.622", "21M.747", "24.115"]
 ["21M.623", "24.947", "ES.113"]

In [28]:
interpret_course_attr(taken_classes_sem, "title")

8-element Vector{Vector{String}}:
 ["Calculus", "Introduction to Computer Science Programming in Python", "Oral Communication", "Introductory Biology", "Physics I"]
 ["Calculus", "Introduction to Computational Thinking and Data Science", "Fundamentals of Programming", "Mathematics for Computer Science"]
 ["Linear Algebra", "Introduction to Algorithms", "Introduction to Probability"]
 ["Networks", "Advances in Computer Vision", "Physics II"]
 ["Dynamical System Modeling and Control Design", "Introduction to Machine Learning", "Computer Graphics"]
 ["American Urban History", "Introduction to Solid-State Chemistry", "Computers and Social Change"]
 ["Physical Improvisation: Scores and Structures", "Talking and Dancing", "Philosophy and Time"]
 ["Physical Improvisation: Bodies in Motion", "Language Disorders in Children", "Ancient Greek Philosophy and Mathematics"]

In [29]:
prerequisite_expressions[subjectid_to_index["16.84"]]

PreReq(CourseNode("16.84"), ORNode(CourseNode("6.4200"), CourseNode("''permissionofinstructor''")))

In [30]:
sems_hours = interpret_course_hours(taken_classes_sem)
[sum(sem_hours) for sem_hours in sems_hours]

8-element Vector{Float64}:
 47.56
 42.29
 32.11
 30.78
 30.71
 17.89
 13.899999999999999
 11.74

In [35]:
function get_course_path(major::String, alpha=.8)
    model = create_model(GRB_ENV)
    course_var = add_course_semester_variables!(model)
    set_objective_func(model, course_var, alpha)
    add_general_requirements(model, course_var)
    add_major_requirements(major, model, course_var)
    add_prereq_constraints(model, course_var)
    optimize!(model)
    taken_classes_sem = [[ind for (ind, val) in enumerate(sem) if val == 1] for sem in eachcol(value.(course_var))]
    #return interpret_course_nums(taken_classes_sem)
    return interpret_course_attr(taken_classes_sem ,"title")
end

get_course_path (generic function with 2 methods)

In [36]:
get_course_path("18am")

MethodError: MethodError: no method matching zero(::Type{Any})
This error has been manually thrown, explicitly, so the method may exist but be intentionally marked as unimplemented.

Closest candidates are:
  zero(::Type{Union{Missing, T}}) where T
   @ Base missing.jl:105
  zero(!Matched::Type{Union{}}, Any...)
   @ Base number.jl:310
  zero(!Matched::Type{Dates.DateTime})
   @ Dates ~/.julia/juliaup/julia-1.11.2+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Dates/src/types.jl:458
  ...


# Web Stuff

In [ ]:
Pkg.add("Genie")

In [ ]:
using Genie, Genie.Renderer.Html, Genie.Requests

form = """
<form action="/" method="POST" enctype="multipart/form-data">
  <input type="text" name="name" value="" placeholder="What's your name?" />
  <input type="submit" value="Greet" />
</form>
"""

route("/") do
  html(form)
end

route("/", method = POST) do
  "Hello $(postpayload(:name, "Anon"))"
end

up()